In [90]:
import pandas as pd
from pandas import DataFrame as df
import xml.etree.ElementTree as ET

In [91]:
buy_from_customer = df(columns=["date",
                        "phone_number",
                        "notebook",
                        "tablet",
                        "phone",
                        "spare_part",
                        "conversation",
                        "customer_price",
                        "our_price",
                        "recycle",
                        "customer_decision"])
buy_from_customer

,date,phone_number,notebook,tablet,phone,spare_part,conversation,customer_price,our_price,recycle,customer_decision


In [136]:
tree = ET.parse('output.xml')
root = tree.getroot()
r = root.findall("Lead")

def make_dict_of_leads(root=root):
    dict_of_leads = dict();
    for lead in root.iter('Lead'):
        id = lead.attrib["id"]

        pulpy = ET.fromstring(lead.attrib["text"])
        source_sentence = pulpy.find("b").find("s");
        text = "";
        for i in source_sentence.itertext():
            text += i;
        dict_of_leads[id] = text;
    return dict_of_leads;

make_dict_of_leads()["46"]

'89140580517 продажа айфон6s на выезде Ляпидевского 12.'

In [135]:
# todo: помнить о тексте лида. Там выделены факты прямо в разметке - полезно при выводе информации в веб-интерфейсе

def compare_facts_to_leads(root=root):
    facts_grouped_by_lead = dict()

    for i in root.find("document").find('facts'):
        lead_id = i.attrib['LeadID']
        if facts_grouped_by_lead.get(lead_id):
            facts_grouped_by_lead[lead_id].append(i)
        else:
            facts_grouped_by_lead[lead_id] = [i]
    return facts_grouped_by_lead

facts = compare_facts_to_leads()
for lead in facts:
    elems = facts[lead]
    one_sentence = lead
    for e in elems:
        for i in e:
            one_sentence += " | " + i.attrib["val"]
        one_sentence += " | " + e.tag
    print(one_sentence)

46 | 89140580517 | CustomerPhone | АЙФОН6S | Phone
24 | НОУТБУК | Notebook
53 | 89140580517 | CustomerPhone | ACER | Notebook
18 | PAWN | Pawn
10 | BUY | 17 | Buy | 89140580517 | CustomerPhone
28 | 89140580517 | CustomerPhone | АСЕР | Notebook
54 | 89140580517 | CustomerPhone
30 | 89140580517 | CustomerPhone | АСЕР | Notebook
3 | 89140580517 | CustomerPhone | УУ | CustomerPlace | НОУТБУК | Notebook | BUY | Buy
19 | PAWN | НОУТБУК | Pawn | НОУТБУК | АСЕР | Notebook | 89140580517 | CustomerPhone
55 | 89140580517 | CustomerPhone | BUY | Buy
21 | PAWN | АСЕР | Pawn | 89140580517 | CustomerPhone | АСЕР | Notebook
29 | 89140580517 | CustomerPhone | АСЕР | Notebook
6 | 89140580517 | CustomerPhone | BUY | Buy | ТЕЛЕФОН | Phone
22 | PAWN | СОНИ | Pawn | СОНИ | ХПЕРИЯ З3 | Phone | 89140580517 | CustomerPhone
7 | 89140580517 | CustomerPhone | BUY | Buy | ТЕЛЕФОН | Phone
40 | ТЕЛЕФОН | SAMSUNG | А5 | Phone | 89140580517 | CustomerPhone
49 | 89140580517 | CustomerPhone | ЯКУТИЯ | CustomerPlace
16 |